In [1]:
import pandas as pd
import re
from functools import partial

In [2]:
comments = pd.read_csv('comments.csv').drop('Unnamed: 0', axis=1)
word_cnt = pd.read_csv('word_freq.csv').drop('Unnamed: 0', axis=1)
carbrands_models = pd.read_csv('carmodels_brands.csv')

In [3]:
#getting all the unique brands 
car_brands = list(set(carbrands_models['Brand']))
car_brands = [re.compile(r'[,.]').sub('', i) for i in car_brands]
print("no of brands", len(car_brands))
wordfreq_dict = dict(word_cnt[['word', 'frequency']].values)
missing_brands = set(car_brands) - set(wordfreq_dict.keys())
print("missing brands:", missing_brands)

no of brands 40
missing brands: {'hyndai kia', 'toyata', 'mercedes benz', 'mercedes-benz'}


In [4]:
for brand in missing_brands:
    try:       
        for i in brand.split(' '):
            print([word for word in list(wordfreq_dict.keys()) if i in str(word)])
        for i in brand.split('-'):
            print([word for word in list(wordfreq_dict.keys()) if i in str(word)])
    except Exception as e:
        print(brand)
# considering hyndai kia as kia, mercedes-benz as benz , mercedes benz as benz,toyata as toyota
mis_brand_handle = {'hyndai kia': 'kia', 'mercedes-benz': 'benz', 'mercedes benz': 'benz', 'toyata': 'toyota'}
car_brands = [mis_brand_handle[b] if b in list(mis_brand_handle.keys()) else b for b in car_brands]

[]
['kia', 'kias', 'nokkia']
[]
[]
[]
['mercedes']
['benz', 'benzes', 'benzs']
[]
[]
['mercedes']
['benz', 'benzes', 'benzs']


In [5]:
mis_brand_handle_t = {'kia': ['hyndai kia'], 'benz': ['mercedes-benz', 'mercedes benz' ], 'toyota': ['toyota']}

In [6]:
def get_lift_value(a_brand, b_brand, comments, mis_brand_handle_t):
    """
    returns the lift value for given two brands 
    """
    p_a_b = 0
    p_a =0
    p_b =0
    for comment in comments:
        p_a_tmp =0
        p_b_tmp =0
        if type(comment) == float:
            continue
        else:
            for word in comment.split():
                word = re.sub(r'[^\w]', ' ', word).strip().lower()
                if " " in word:
                    for w in word.split():
                        w = re.sub(r'[^\w]', ' ', w).strip().lower()
                        if a_brand in w or w in a_brand:
                            p_a_tmp+= 1
                        if b_brand in w or w in b_brand:
                            p_b_tmp+=1
                        if a_brand in mis_brand_handle_t:
                            p_a_tmp = p_a_tmp + sum([1 for brand in mis_brand_handle_t[a_brand] 
                                                     if brand in w or word in brand])
                        if b_brand in mis_brand_handle_t:
                            p_b_tmp = p_b_tmp + sum([1 for brand in mis_brand_handle_t[b_brand] 
                                                     if brand in w or w in brand])
                            
                else:
                    if a_brand in word or word in a_brand:
                        p_a_tmp+= 1
                    if b_brand in word or word in b_brand:
                        p_b_tmp+=1
                    if a_brand in mis_brand_handle_t:
                        p_a_tmp = p_a_tmp + sum([1 for brand in mis_brand_handle_t[a_brand] 
                                                 if brand in word or word in brand])
                    if b_brand in mis_brand_handle_t:
                        p_b_tmp = p_b_tmp + sum([1 for brand in mis_brand_handle_t[b_brand] 
                                                 if brand in word or word in brand])
        if p_a_tmp>0:
            p_a+=1
        if p_b_tmp>0:
            p_b+=1
        if p_a_tmp >0 and p_b_tmp > 0:
            p_a_b+=1
    if p_a > 0 and p_b >0:
        return  (p_a_b / len(comments)) / ((p_a/len(comments)) * (p_b/len(comments)))
    else:
        return 0

In [7]:
comments = comments['comments']

In [8]:
# fixing the arguments comemnts and mis_brand_handle_t for function get_lift_value
get_lift_value_p = partial(get_lift_value, comments=comments, mis_brand_handle_t= mis_brand_handle_t)

In [9]:
# car_brands = car_brands[:5]

In [10]:
%%time
import numpy as np
lift_values = []
for i_1, brand_1 in dict(list(enumerate(car_brands))).items():
    lift_value_tmp = []
    for i_2, brand_2 in dict(list(enumerate(car_brands))).items():
        if i_2 == i_1:
            lift_value_tmp.append(1)
        elif i_2 < i_1:
            lift_value_tmp.append(lift_values[i_2][i_1])
        else:
            lift_value_tmp.append(get_lift_value_p(brand_1, brand_2))
    lift_values.append(lift_value_tmp)  

CPU times: total: 8min 37s
Wall time: 8min 38s


In [14]:
# lift_values = pd.DataFrame(lift_values, columns=car_brands, index=car_brands)
lift_values.to_csv('lift_values.csv')

In [12]:
# df = pd.DataFrame(columns=car_brands, index=car_brands)
# df[car_brands] = 0
# df

In [13]:
# df = df.apply(lambda row: row.name==row.)